# Multinational Retail Data Centralisation

This notebook is used to interactively work with the classes and the data returned so that development is easier. For example, interacting with the DataFrame to understand the data in the database, to create methods for cleaning.

In [1]:
import pandas as pd
from database_utils import DatabaseConnector
from data_extraction import DataExtractor

connector = DatabaseConnector()
extractor = DataExtractor(connector)

## Fetch DataFrame from table name

Using connector to find table names, and then using extractor to produce a DataFrame of a specific table.

In [ ]:
connector.list_db_tables()

In [ ]:
df = extractor.read_rds_table("legacy_users")
df.head(5)

In [ ]:
df.info()

## Cleaning user data

Interactively attempting to clean the data in the user table, so that this can be implemented in the DataCleaning class.

In [ ]:
# Convert object columns to their respective type
df = df.astype(
    {
        "first_name": "string",
        "last_name": "string",
        "company": "string",
        "email_address": "string",
        "address": "string",
        "country_code": "string",
        "country": "string",
        "user_uuid": "string"
    }
)

df.dtypes

In [ ]:
# Convert object date columns to the datetime type
date_format = "%Y-%m-%d"
df.date_of_birth = pd.to_datetime(df.date_of_birth, errors='coerce', format=date_format)
df.join_date = pd.to_datetime(df.join_date, errors='coerce', format=date_format)

df.info()

In [ ]:
# We can confirm actual user entries among bad data by their UUID
from re import search
uuid_regex = r'^[0-9A-Za-z]{8}-[0-9A-Za-z]{4}-4[0-9A-Za-z]{3}-[89ABab][0-9A-Za-z]{3}-[0-9A-Za-z]{12}$'

good_uuid = "93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8"
bad_uuid = "AS45323"

match_good = search(uuid_regex, good_uuid)
match_bad = search(uuid_regex, bad_uuid)
match_good, match_bad

In [ ]:
# pandas suggest using pd.NA over numpy.nan for string type columns
df.loc[~df.user_uuid.str.match(uuid_regex, na=False), 'user_uuid'] = pd.NA

df[df.user_uuid.isna()].head()

In [ ]:
# We can see some rows have incorrect country code GB as GGB
df.country_code.value_counts()

In [ ]:
df.country_code = df.country_code.replace("GGB", "GB")
df.country_code.value_counts()

In [ ]:
df.phone_number.head(50)

In [26]:
import phonenumbers
import re

def parse_phone_number(phone: str, region: str):
    # Clean the phone number by removing (0), extensions, and other unnecessary characters
    phone = re.sub(r'\(0\)', '', phone)  # Remove (0)
    phone = phone.replace("(", "").replace(")", "")  # Remove parentheses
    phone = re.sub(r'x.*$', '', phone)  # Remove extensions (e.g., x1234)
    phone = re.sub(r'[^\d+]', '', phone)  # Remove non-numeric characters except for +

    try:
        # Attempt to parse the number with the phonenumbers library
        # If no '+' sign, assume it's a local number and use the default region
        if not phone.startswith('+'):
            parsed_number = phonenumbers.parse(phone, region)
        else:
            parsed_number = phonenumbers.parse(phone)

        # Format the parsed number in international format
        return phonenumbers.format_number(parsed_number, phonenumbers.PhoneNumberFormat.INTERNATIONAL)

    except phonenumbers.phonenumberutil.NumberParseException:
        return None

df.phone_number = df.apply(
    lambda row: parse_phone_number(row['phone_number'], row['country_code']), axis=1
) # type: ignore


In [ ]:
df.loc[df.country_code == "DE"].head(10)

In [ ]:
# drop any null rows
df.replace("NULL", pd.NA, inplace=True)
df = df.dropna(how='any', axis='index')
df.info()

## Cleaning card data

In [ ]:
# Get PDF data as DataFrame
pdf_dfs = extractor.retrieve_pdf_data("https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf")

pdf_dfs.head(5)

In [ ]:
# Some strange columns that got detected by tabular that need dropping
cleaned_df = pdf_dfs.drop(columns=['card_number expiry_date', 'Unnamed: 0'])
cleaned_df.head(5)

In [ ]:
# replace NULL with pandas na
cleaned_df = cleaned_df.replace("NULL", pd.NA)
cleaned_df.head(5)

In [ ]:
# remove non-numerical characters from card number
cleaned_df.card_number = cleaned_df.card_number.replace(r'[^0-9]+', '', regex=True)
cleaned_df.head(5)

In [ ]:
# replace empty card numbers with pandas NA
cleaned_df.card_number = cleaned_df.card_number.replace('', pd.NA, regex=True)
cleaned_df.head(5)

In [ ]:
# convert card number column to int, with coerce so any failed conversions are null
cleaned_df.card_number = pd.to_numeric(cleaned_df.card_number, errors='coerce').astype("Int64")
cleaned_df.head(5)

In [ ]:
# change card_provider column to string
cleaned_df.card_provider = cleaned_df.card_provider.astype("string")
cleaned_df.head(5)

In [101]:
# convert expiry date column to datetime
cleaned_df.expiry_date = pd.to_datetime(cleaned_df.expiry_date, format="%m/%y", errors='coerce')

## API requests

Using Json and requests lib to get data from the API

In [ ]:
import requests

# retrieve_store_url, retrieve_store_count_url, header (x-api-key, Content-Type)
api_config = extractor.load_api_config()
api_config

In [ ]:
# Get the number of stores from the API
response = requests.get(api_config["retrieve_store_count_url"], headers=api_config["header"])
data = response.json()
data["number_stores"]

In [ ]:
specific_store = api_config["retrieve_store_url"] + str(0)
response = requests.get(specific_store, headers=api_config["header"])
data = response.json()
data

In [12]:
from time import sleep

number_of_stores = 451
store_jsons = []
for i in range(number_of_stores):
    specific_store = api_config["retrieve_store_url"] + str(i)
    response = requests.get(specific_store, headers=api_config["header"])
    store_jsons.append(response.json())
    sleep(0.05) # rate limit

In [ ]:
len(store_jsons)

In [ ]:
store_df = pd.DataFrame(store_jsons)
store_df.head(5)

## Clean store data

In [ ]:
# Replace N/A string with pandas NA
cleaned_store_df = store_df.replace('N/A', pd.NA)
cleaned_store_df.head(5)

In [ ]:
# Replace pythonic None from json response to pandas NA
cleaned_store_df = cleaned_store_df.replace([None], pd.NA)
cleaned_store_df.head(5)

In [ ]:
# Noticed the lat column appears useless, since latitude exists with values. So drop it
cleaned_store_df = cleaned_store_df.drop(columns=['lat'])
cleaned_store_df.head(5)

In [ ]:
# Change types
cleaned_store_df = cleaned_store_df.astype(
    {
        "address": "string",
        "locality": "string",
        "store_code": "string",
        "store_type": "string",
        "country_code": "string",
        "continent": "string"
    }
)
cleaned_store_df.dtypes

In [ ]:
# Remove letters from any numbers in staff numbers
cleaned_store_df.staff_numbers = cleaned_store_df.staff_numbers.replace(r'[^0-9]+', '', regex=True)
cleaned_store_df.staff_numbers = cleaned_store_df.staff_numbers.replace('', pd.NA, regex=True)
cleaned_store_df.staff_numbers = pd.to_numeric(cleaned_store_df.staff_numbers, errors='coerce').astype("Int64")
cleaned_store_df.dtypes

In [ ]:
# Convert long/latitude to float
cleaned_store_df.longitude = pd.to_numeric(cleaned_store_df.longitude, errors='coerce').astype("float")
cleaned_store_df.latitude = pd.to_numeric(cleaned_store_df.latitude, errors='coerce').astype("float")
cleaned_store_df.dtypes

In [ ]:
# Convert opening date
date_format = "%Y-%m-%d"
cleaned_store_df.opening_date = pd.to_datetime(cleaned_store_df.opening_date, errors="coerce", format=date_format)
cleaned_store_df.dtypes

In [ ]:
# Some strange values in here. We only want GB, DE, US
cleaned_store_df.country_code.value_counts()

In [ ]:
# Only get rows whose country_code is a valid one
country_codes = ["GB", "DE", "US"]
mask = cleaned_store_df.country_code.isin(country_codes)
cleaned_store_df = cleaned_store_df[mask]
cleaned_store_df.country_code.value_counts()

In [ ]:
# Again some strange values here, time to replace
cleaned_store_df.continent.value_counts()

In [ ]:
cleaned_store_df.loc[:, 'continent'] = cleaned_store_df['continent'].str.replace("ee", "")
cleaned_store_df.continent.value_counts()

In [ ]:
cleaned_store_df = cleaned_store_df.dropna(how='any', axis='index')
cleaned_store_df.info()

In [ ]:
cleaned_store_df.head(50)